<h1> In diesem Nodebook werden noch mal die Daten untersucht </h1>


In [27]:
!pip install shap


   ---------------------------------------- 0.0/545.2 kB ? eta -:--:--
   ---------------------------------------- 545.2/545.2 kB 6.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import shap
import matplotlib.pyplot as plt
import seaborn as sns

<p>Modellierunsdaten ladne</p>

In [3]:
df = pd.read_csv(r"modellierung_02.csv", index_col=0)
df

,Deutschland (Positiv),Deutschland (Negativ),Solar,Wind Offshore,Wind Onshore,Residuallast,wind_speed_100m,wind_gusts_10m,cloudcover,temperature_2m,...,precipitation,_sin_year,_cos_year,_sin_week,_cos_week,_sin_day,_cos_day,load forecast error Deutschland,gradient Deutschland,Feiertagsindex
timestamp,,,,,,,,,,,,,,,,,,,,,
2022-06-30 22:00:00+00:00,390.620,350.816,3.0,1317.0,13862.0,37257.0,22.50,22.700,100.0,18.100,...,7.000,0.025818,-0.999667,0.433884,-0.900969,-0.500000,0.866025,1044.77,-107.77,False
2022-06-30 22:15:00+00:00,162.736,263.656,3.0,1459.0,13702.0,37019.0,22.65,22.775,100.0,17.300,...,6.525,0.025818,-0.999667,0.433884,-0.900969,-0.442289,0.896873,230.69,-1092.17,False
2022-06-30 22:30:00+00:00,928.544,676.444,3.0,1480.0,13340.0,36835.0,22.80,22.850,100.0,16.500,...,6.050,0.025818,-0.999667,0.433884,-0.900969,-0.382683,0.923880,171.95,-695.35,False
2022-06-30 22:45:00+00:00,1212.684,825.452,3.0,1428.0,12910.0,36779.0,22.95,22.925,100.0,15.700,...,5.575,0.025818,-0.999667,0.433884,-0.900969,-0.321439,0.946930,122.87,-392.45,False
2022-06-30 23:00:00+00:00,1402.408,963.996,3.0,1352.0,12591.0,37140.0,23.10,23.000,100.0,14.900,...,5.100,0.025818,-0.999667,0.433884,-0.900969,-0.258819,0.965926,-31.08,-563.35,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-17 21:45:00+00:00,393.812,397.516,7.0,2849.0,2403.0,40560.0,11.10,18.175,100.0,-2.550,...,0.000,0.288482,0.957485,-0.433884,-0.900969,-0.555570,0.831470,0.00,0.00,False
2025-01-17 22:00:00+00:00,470.056,646.360,6.0,2893.0,2499.0,41371.0,10.40,18.000,100.0,-2.600,...,0.000,0.288482,0.957485,-0.433884,-0.900969,-0.500000,0.866025,0.00,0.00,False
2025-01-17 22:15:00+00:00,786.280,721.776,6.0,2920.0,2553.0,40780.0,9.55,17.375,100.0,-2.525,...,0.000,0.288482,0.957485,-0.433884,-0.900969,-0.442289,0.896873,0.00,0.00,False


In [9]:
for i in df.columns:
    print(i)
    print(df[i].describe())

Deutschland (Positiv)
count    89476.000000
mean       696.191977
std        397.499785
min          0.000000
25%        412.130000
50%        616.508000
75%        891.566000
max       4270.820000
Name: Deutschland (Positiv), dtype: float64
Deutschland (Negativ)
count    89476.000000
mean       657.800190
std        382.401633
min          0.000000
25%        385.495000
50%        584.836000
75%        850.596000
max       4350.640000
Name: Deutschland (Negativ), dtype: float64
Solar
count    89476.000000
mean      6483.724909
std      10166.759933
min          1.000000
25%          4.000000
50%         68.000000
75%      10051.250000
max      47066.000000
Name: Solar, dtype: float64
Wind Offshore
count    89476.000000
mean      2829.372815
std       1865.745695
min          0.000000
25%       1151.000000
50%       2639.000000
75%       4398.000000
max       7640.000000
Name: Wind Offshore, dtype: float64
Wind Onshore
count    89476.000000
mean     12634.649984
std       9997.105654
m

In [13]:
#XGBoost für Positve Regelleistung, also Deutschland (Positiv)
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [29]:
def get_test_train(daten):
    # Beide Zielgrößen als DataFrame gemeinsam behandeln
    y_all = daten[["Deutschland (Positiv)", "Deutschland (Negativ)"]]
    X = daten.drop(columns=["Deutschland (Positiv)", "Deutschland (Negativ)"])
    # Split einmal durchführen
    X_train, X_test, y_train, y_test = train_test_split(X, y_all, test_size=0.2, random_state=42)

    # Jetzt einzeln extrahieren
    y_pos_train = y_train["Deutschland (Positiv)"].values
    y_neg_train = y_train["Deutschland (Negativ)"].values
    y_pos_test = y_test["Deutschland (Positiv)"].values
    y_neg_test = y_test["Deutschland (Negativ)"].values
    return y_pos_train, y_neg_train, y_pos_test, y_neg_test, X_train, X_test                

In [30]:
def xgboost(daten):
    # Modell erstellen (ähnlich wie RandomForestRegressor)
    model_pos_xgboost = XGBRegressor(
        n_estimators=300,
        learning_rate=0.1,
        max_depth=30,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1
    )
    model_neg_xgboost = model_pos_xgboost

    y_pos_train, y_neg_train, y_pos_test, y_neg_test, X_train, X_test = get_test_train(daten)
    

    model_berechnen("positiv", y_pos_train, X_train, y_pos_test, X_test)
    model_berechnen("negativ", y_neg_train, X_train, y_neg_test, X_test)



In [ ]:
def model_berechnen(name, y_train, X_train, y_test, X_test): 
    print(f"Model {name} trainieren")
    #hier wird das Model drin trainiert
    model_xgboost = XGBRegressor(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=30,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1
    )
     # Modell trainieren
    model_xgboost.fit(X_train, y_train)
    y_pred = model_xgboost.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"MSE {name}:", mse)
    print(f"R² Score {name}:", r2)
    print(f"RMSE positiv {name}:", np.sqrt(mse))


        # --- 3. SHAP-Werte berechnen ---
    explainer = shap.Explainer(model_xgboost)
    shap_values = explainer(X_train)

    # --- 4. Summary-Plot (wichtige Features) ---
    shap.summary_plot(shap_values, X_train)

    # --- 5. Dependence-Plot für ein Feature (z. B. feature_2) ---
    shap.dependence_plot("feature_2", shap_values.values, X_train)

    # --- 6. Analyse der Unsicherheit in Tails ---
    # => Mittlere absolute SHAP-Werte nach Quantilgruppen von feature_2
    X_train["shap_abs"] = np.abs(shap_values[:, X_train.columns.get_loc("feature_2")].values)
    X_train["quantile"] = pd.qcut(X_test["feature_2"], q=10)

    shap_by_quantile = X_train.groupby("quantile")["shap_abs"].mean()

    plt.figure(figsize=(10, 4))
    shap_by_quantile.plot(kind="bar", color="gray", edgecolor="black")
    plt.ylabel("Mittlerer |SHAP-Wert| von feature_2")
    plt.title("Einfluss von feature_2 über verschiedene Tails")
    plt.tight_layout()
    plt.show()


In [ ]:
xgboost(df)

Model positiv trainieren
MSE positiv: 49525.39644333587
R² Score positiv: 0.6758802627130597
RMSE positiv positiv: 222.54302155613837


In [1]:
print("Test TEst Teset")

Test TEst Teset
